In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
username = 'vigrose'
password_edw = 'Slytherin~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
ov_me_sql = '''
    SELECT medical_education_number,type, medical_school, medical_school_state FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [5]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [6]:
MORE_SCHOOL_QUERY = \
    """
    SELECT
    PARTY_ID AS PARTY_ID_SCHOOL,
    KEY_VAL AS SCHOOL_ID
    FROM
    AMAEDW.PARTY_KEY
    WHERE
    KEY_TYPE_ID = 23
    """
more_school_ids = pd.read_sql(con=AMAEDW, sql=MORE_SCHOOL_QUERY)

In [7]:
SCHOOL_QUERY = \
    """
    SELECT
    P.PARTY_ID AS PARTY_ID_SCHOOL,
    P.KEY_VAL AS SCHOOL_ID,
    C.STATE_ID,
    C.COUNTRY_ID
    FROM
    AMAEDW.PARTY_KEY P, AMAEDW.POST_CD C, AMAEDW.PARTY_ADDR A
    WHERE
    P.KEY_TYPE_ID = 23
    AND
    P.PARTY_ID = A.PARTY_ID
    AND 
    A.POST_CD_ID = C.POST_CD_ID
    """
school_ids = pd.read_sql(con=AMAEDW, sql=SCHOOL_QUERY)

In [8]:
MED_SCHOOL_QUERY = \
    """
    SELECT
    SCH_PARTY_ID,
    STU_PARTY_ID AS PARTY_ID,
    GRAD_DT,
    STS_TYPE_ID,
    DEGREE_CD,
    GRAD_CONF_IND
    FROM
    AMAEDW.SCHOOL_ATT
    WHERE
    THRU_DT is null
    """
med_school = pd.read_sql(con=AMAEDW, sql=MED_SCHOOL_QUERY)

In [9]:
GME_QUERY = \
    """
    SELECT
    G.PARTY_HOSPITAL_ID,
    G.PERSON_PARTY_ID AS PARTY_ID,
    G.BEGIN_DT,
    G.END_DT,
    G.PRIM_SPEC_ID,
    G.SEC_SPEC_ID,
    G.TRAIN_TYPE,
    G.CONF_STS_ID,
    G.GME_STS_TYPE_CD,
    C.STATE_ID,
    C.COUNTRY_ID
    FROM
    AMAEDW.EDW_GME G, AMAEDW.POST_CD C, AMAEDW.PARTY_ADDR A
    WHERE
    G.THRU_DT is null
    AND
    G.PARTY_HOSPITAL_ID = A.PARTY_ID
    AND 
    A.POST_CD_ID = C.POST_CD_ID
    """
gme = pd.read_sql(con=AMAEDW, sql=GME_QUERY)

In [10]:
YEAR_QUERY = \
    """
    SELECT
    PARTY_ID, 
    PROG_YEAR,
    PROG_GRAD_YR
    FROM
    AMAEDW.MED_PROF
    WHERE
    THRU_DT is null
    """
gme_year = pd.read_sql(con=AMAEDW, sql=YEAR_QUERY)

In [11]:
ORG_QUERY = \
    """
    SELECT
    PARTY_ID AS PARTY_HOSPITAL_ID,
    ORG_NM, 
    THRU_DT
    FROM
    AMAEDW.ORG_NM
    """
org_names = pd.read_sql(con=AMAEDW, sql=ORG_QUERY)

In [12]:
org_names.THRU_DT = pd.to_datetime(org_names.THRU_DT)

In [13]:
org_names = org_names.sort_values('THRU_DT').drop_duplicates('PARTY_HOSPITAL_ID', keep='last')

In [14]:
school_ids = school_ids.drop_duplicates()

In [15]:
gme_singular = gme.drop_duplicates('PARTY_ID')

In [16]:
extra = more_school_ids[~more_school_ids.SCHOOL_ID.isin(school_ids.SCHOOL_ID)]
school_ids = pd.concat([school_ids, extra])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [17]:
test = pd.merge(party_ids, ov_me[['medical_education_number', 'type']], left_on='ME', right_on='medical_education_number')
school_info = pd.merge(school_ids, org_names, left_on='PARTY_ID_SCHOOL', right_on='PARTY_HOSPITAL_ID', how='left').drop_duplicates()
med_with_info = pd.merge(med_school, school_info, left_on='SCH_PARTY_ID', right_on='PARTY_ID_SCHOOL', how='left').drop_duplicates()
gme_with_info = pd.merge(gme_singular, org_names, on='PARTY_HOSPITAL_ID', how='left').drop_duplicates()

In [18]:
date_list = []
for row in med_with_info.itertuples():
    try:
        new_date = datetime.datetime.strptime(str(row.GRAD_DT), '%Y-%m-%d')
    except:
        new_date = None
    date_list.append(new_date)
med_with_info['GRAD_DATE'] = date_list
med_with_info = med_with_info.sort_values('GRAD_DATE')

In [19]:
countries = []
statuses = []
for row in med_with_info.itertuples():
    country = 0
    status = 0
    if row.COUNTRY_ID == 6705:
        country = 1
    if row.STS_TYPE_ID == 9:
        status = 2
    if row.STS_TYPE_ID == 54:
        status = 1
    countries.append(country)
    statuses.append(status)
med_with_info['COUNTRY'] = countries
med_with_info['STATUS'] = statuses
med_with_info = med_with_info.sort_values(['STATUS','COUNTRY','GRAD_DATE'])

In [20]:
cc = med_with_info.drop_duplicates('PARTY_ID', keep=False)
DUPES = med_with_info[med_with_info.duplicated('PARTY_ID', keep=False)]
plz = DUPES.drop_duplicates('PARTY_ID', keep='last')
med_with_info_2 = pd.concat([plz, cc])

In [21]:
ALL = pd.merge(test, gme_year, on='PARTY_ID', how='left')
ALL = pd.merge(ALL, med_with_info_2, on='PARTY_ID',how='left')
ALL = pd.merge(ALL, gme_with_info, on='PARTY_ID', how='left', suffixes = ['_SCH', '_GME'])

In [28]:
ALL.columns

Index(['PARTY_ID', 'ME', 'medical_education_number', 'type', 'PROG_YEAR',
       'PROG_GRAD_YR', 'SCH_PARTY_ID', 'GRAD_DT', 'STS_TYPE_ID', 'DEGREE_CD',
       'GRAD_CONF_IND', 'COUNTRY_ID_SCH', 'PARTY_ID_SCHOOL', 'SCHOOL_ID',
       'STATE_ID_SCH', 'PARTY_HOSPITAL_ID_SCH', 'ORG_NM_SCH', 'THRU_DT_SCH',
       'GRAD_DATE', 'COUNTRY', 'STATUS', 'PARTY_HOSPITAL_ID_GME', 'BEGIN_DT',
       'END_DT', 'PRIM_SPEC_ID', 'SEC_SPEC_ID', 'TRAIN_TYPE', 'CONF_STS_ID',
       'GME_STS_TYPE_CD', 'STATE_ID_GME', 'COUNTRY_ID_GME', 'ORG_NM_GME',
       'THRU_DT_GME'],
      dtype='object')

In [31]:
ALL.groupby(['type','PROG_GRAD_YR']).count()

PARTY_ID       ME  medical_education_number  \
type      PROG_GRAD_YR                                                
Physician 0.0            1126963  1126963                   1126963   
          1.0               6926     6926                      6926   
          2.0              14556    14556                     14556   
          3.0              20466    20466                     20466   
          4.0              15773    15773                     15773   
          5.0              11394    11394                     11394   
          6.0               6766     6766                      6766   
          7.0               2312     2312                      2312   
          8.0                746      746                       746   
          9.0                296      296                       296   
          10.0                81       81                        81   
          11.0                33       33                        33   
          12.0                14       14                        14   
          13.0                 5        5                         5   
          14.0                 2        2                         2   
          19.0                 1        1                         1   
          21.0                 1        1                         1   
          99.0                 1        1                         1   
Resident  0.0             103245   103245                    103245   
          1.0              10200    10200                     10200   
          2.0               9228     9228                      9228   
          3.0               5618     5618                      5618   
          4.0               1793     1793                      1793   
          5.0                572      572                       572   
          6.0                191      191                       191   
          7.0                 65       65                        65   
          8.0                 21       21                        21   
          9.0                  6        6                         6   
          11.0                 4        4                         4   
          13.0                 1        1                         1   
          15.0                 1        1                         1   
          28.0                 1        1                         1   

                        PROG_YEAR  SCH_PARTY_ID  GRAD_DT  STS_TYPE_ID  \
type      PROG_GRAD_YR                                                  
Physician 0.0             1126963       1126963  1126963      1126963   
          1.0                6926          6926     6926         6926   
          2.0               14556         14556    14556        14556   
          3.0               20466         20466    20466        20466   
          4.0               15773         15773    15773        15773   
          5.0               11394         11394    11394        11394   
          6.0                6766          6766     6766         6766   
          7.0                2312          2312     2312         2312   
          8.0                 746           746      746          746   
          9.0                 296           296      296          296   
          10.0                 81            81       81           81   
          11.0                 33            33       33           33   
          12.0                 14            14       14           14   
          13.0                  5             5        5            5   
          14.0                  2             2        2            2   
          19.0                  1             1        1            1   
          21.0                  1             1        1            1   
          99.0                  1             1        1            1   
Resident  0.0              103245        103245   103245       103245   
          1.0               10200         10200    10200        10200   
   

In [39]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
all_mes = list(ALL.ME)
us_schools = list(ALL[(ALL.COUNTRY_ID_SCH==6705)].ME)
us_gmes = list(ALL[(ALL.COUNTRY_ID_GME==6705)].ME)
elements = [
    {'data_element': 'Medical School Name',
     'element':'ORG_NM_SCH',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Medical School State',
     'element':'STATE_ID_SCH',
    'null': ['',-1],
    'universe': us_schools},
    {'data_element': 'Medical School Country',
     'element':'COUNTRY_ID_SCH',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'Medical School Grad Date',
     'element':'GRAD_DT',
    'null': ['',-1, datetime.date(1900, 1, 30)],
    'universe': all_mes},
    {'data_element': 'Education Status',
     'element':'STS_TYPE_ID',
    'null': ['',-1,61,43],
    'universe': all_mes},
    {'data_element': 'Medical Degree',
     'element':'DEGREE_CD',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'GME Institution',
     'element':'ORG_NM_GME',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'GME Institution State',
     'element':'STATE_ID_GME',
    'null': ['',-1],
    'universe': us_gmes},
    {'data_element': 'GME Begin Date',
     'element':'BEGIN_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME End Date',
     'element':'END_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Primary Specialty',
     'element':'PRIM_SPEC_ID',
    'null': ['',-1,1883],
    'universe':all_mes},
    {'data_element': 'GME Secondary Specialty',
     'element':'SEC_SPEC_ID',
    'null': ['',-1,1883],
    'universe': all_mes},
    {'data_element': 'GME Training Type',
     'element':'TRAIN_TYPE',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Confirmation Status',
     'element':'CONF_STS_ID',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Status Type',
     'element':'GME_STS_TYPE_CD',
    'null': ['',-1],
    'universe':all_mes},
    {'data_element': 'GME Year in Program',
     'element':'PROG_YEAR',
    'null': ['',-1],
    'universe':all_mes},
    {'data_element': 'GME Post Grad Year',
     'element':'PROG_GRAD_YR',
    'null': ['',-1],
    'universe':all_mes}
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        universe = len(ALL[(ALL.type==typo)&(ALL.ME.isin(element['universe']))])
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        true_complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isin(element['null']))&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [40]:
pd.DataFrame(dict_list).to_csv(f'../../Data/Measurement/Education_Completeness_{str(datetime.date.today())}.csv', index=False)

In [ ]:
datetime.datetime.strptime(today, '%Y-%m-%d')

In [28]:
ALL[(ALL.type=='Student')&(ALL.GRAD_DT<datetime.date(2022,8,16))&~(ALL.GME_STS_TYPE_CD.isna())]

,PARTY_ID,ME,medical_education_number,type,PROG_YEAR,PROG_GRAD_YR,SCH_PARTY_ID,GRAD_DT,STS_TYPE_ID,DEGREE_CD,...,END_DT,PRIM_SPEC_ID,SEC_SPEC_ID,TRAIN_TYPE,CONF_STS_ID,GME_STS_TYPE_CD,STATE_ID_GME,COUNTRY_ID_GME,ORG_NM_GME,THRU_DT_GME
71081,6160542,02405180131,02405180131,Student,NaN,NaN,2696235,2018-05-01,17,MD,...,2020-06-30,1724.0,1883.0,R,52.0,COMPLETED,198.0,6705.0,University of Florida College of Medicine,NaT
71747,6161268,02604190366,02604190366,Student,NaN,NaN,2692975,2020-05-01,13,MD,...,2024-06-30,1793.0,1883.0,R,52.0,CURRENT,249.0,6705.0,University of Minnesota Medical School,NaT
71763,6161285,02604190382,02604190382,Student,NaN,NaN,2692975,2020-05-01,13,MD,...,2021-06-30,1724.0,1883.0,R,52.0,CURRENT,248.0,6705.0,Johns Hopkins University School of Medicine,NaT
71797,6161344,02604190391,02604190391,Student,NaN,NaN,2692975,2020-05-01,13,MD,...,2021-06-30,1800.0,1883.0,R,52.0,COMPLETED,239.0,6705.0,University of Texas Southwestern Medical School,NaT
71919,6161495,02802151040,02802151040,Student,NaN,NaN,2855141,2020-05-15,13,MD,...,2023-06-30,1845.0,1883.0,R,52.0,CURRENT,232.0,6705.0,Stanford Hospital and Clinics,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383528,16129144,04975210303,04975210303,Student,NaN,NaN,11474036,2021-05-15,8,DO,...,2024-06-30,1693.0,1883.0,R,52.0,CURRENT,232.0,6705.0,St Agnes Medical Center (Fresno),NaT
1383556,16132427,05175210441,05175210441,Student,NaN,NaN,3533461,2021-05-08,8,DO,...,2024-06-30,1724.0,1883.0,R,52.0,CURRENT,224.0,6705.0,Prisma Health,NaT
1383690,16151651,04177211201,04177211201,Student,NaN,NaN,2848789,2021-05-24,8,DO,...,2029-06-30,1921.0,1883.0,R,52.0,CURRENT,200.0,6705.0,University of Michigan Hospitals and Health Ce...,NaT
1384107,16210406,04178210535,04178210535,Student,NaN,NaN,2701307,2021-05-30,8,DO,...,2024-06-30,1724.0,1883.0,R,52.0,CURRENT,222.0,6705.0,Albany Medical Center,NaT


In [37]:
ALL[(ALL.type=='Resident')&(ALL.END_DT<datetime.date(2021,8,16))&(ALL.GME_STS_TYPE_CD=='COMPLETED ')]

,PARTY_ID,ME,medical_education_number,type,PROG_YEAR,PROG_GRAD_YR,SCH_PARTY_ID,GRAD_DT,STS_TYPE_ID,DEGREE_CD,...,END_DT,PRIM_SPEC_ID,SEC_SPEC_ID,TRAIN_TYPE,CONF_STS_ID,GME_STS_TYPE_CD,STATE_ID_GME,COUNTRY_ID_GME,ORG_NM_GME,THRU_DT_GME
11166,1897541,00514961581,00514961581,Resident,0.0,0.0,2701282,1996-05-01,9,MD,...,1999-06-30,1693.0,1883.0,R,52.0,COMPLETED,232.0,6705.0,Kaiser Permanente Southern California,NaT
19289,1908288,00702990220,00702990220,Resident,0.0,0.0,2778202,1999-05-01,9,MD,...,2002-06-30,1800.0,1883.0,R,52.0,COMPLETED,232.0,6705.0,Stanford Hospital and Clinics,NaT
32509,6155045,03401150268,03401150268,Resident,2.0,3.0,2701300,2016-05-14,9,MD,...,2021-06-30,1679.0,1883.0,R,52.0,COMPLETED,244.0,6705.0,Duke University Hospital,NaT
32520,6155056,03401150373,03401150373,Resident,3.0,3.0,2701300,2016-05-01,9,MD,...,2020-06-30,1769.0,1883.0,R,52.0,COMPLETED,234.0,6705.0,University of New Mexico School of Medicine,NaT
32570,3687523,26404080031,26404080031,Resident,0.0,0.0,2696290,2008-01-01,9,MD,...,2013-06-30,1724.0,1883.0,R,52.0,COMPLETED,239.0,6705.0,Texas Tech University Health Sciences Center a...,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420832,5805489,91338090159,91338090159,Resident,0.0,0.0,2693403,2009-01-01,9,MD,...,2021-07-31,1724.0,1883.0,R,52.0,COMPLETED,200.0,6705.0,Detroit Medical Center Corporation,NaT
1421121,9388058,49537170211,49537170211,Resident,0.0,0.0,2701380,2017-01-01,9,MD,...,2021-06-30,1800.0,1883.0,R,52.0,COMPLETED,239.0,6705.0,University of Texas Medical Branch Hospitals,NaT
1421124,9113897,49659160128,49659160128,Resident,0.0,0.0,3534695,2016-01-01,9,MD,...,2021-06-30,1800.0,1883.0,R,52.0,COMPLETED,239.0,6705.0,University of Texas Medical Branch Hospitals,NaT
1422219,9053973,42201178341,42201178341,Resident,0.0,0.0,2837054,2017-01-01,9,MD,...,2021-06-30,1639.0,1883.0,R,52.0,COMPLETED,239.0,6705.0,Texas Tech University Health Sciences Center a...,NaT


In [45]:
ALL.columns

Index(['PARTY_ID', 'ME', 'medical_education_number', 'type', 'PROG_YEAR',
       'PROG_GRAD_YR', 'SCH_PARTY_ID', 'GRAD_DT', 'STS_TYPE_ID', 'DEGREE_CD',
       'GRAD_CONF_IND', 'COUNTRY_ID_SCH', 'PARTY_ID_SCHOOL', 'SCHOOL_ID',
       'STATE_ID_SCH', 'PARTY_HOSPITAL_ID_SCH', 'ORG_NM_SCH', 'THRU_DT_SCH',
       'GRAD_DATE', 'COUNTRY', 'STATUS', 'PARTY_HOSPITAL_ID_GME', 'BEGIN_DT',
       'END_DT', 'PRIM_SPEC_ID', 'SEC_SPEC_ID', 'TRAIN_TYPE', 'CONF_STS_ID',
       'GME_STS_TYPE_CD', 'STATE_ID_GME', 'COUNTRY_ID_GME', 'ORG_NM_GME',
       'THRU_DT_GME'],
      dtype='object')

In [90]:
ALL.GME_STS_TYPE_CD.unique()

array(['COMPLETED ', 'NO-REPORT ', 'INCOMPLETE', 'CURRENT   ', nan],
      dtype=object)

In [62]:
sts = pd.read_csv('../../Data/Measurement/STATE.csv')
state_ids = list(sts.STATE_ID)

In [96]:
spec = pd.read_csv('../../Data/Measurement/SPECIALTY.csv')

In [100]:
ALL = pd.merge(ALL, spec[['SPEC_CD','SPEC_ID']], left_on='PRIM_SPEC_ID', right_on='SPEC_ID')

In [104]:
with_specs = pd.merge(ALL, ppd[['PRIM_SPEC_CD','ME']], on='ME')

In [107]:
with_specs[(with_specs.PRIM_SPEC_CD!=with_specs.SPEC_CD)&(with_specs.PRIM_SPEC_CD=='US')]

,PARTY_ID,ME,medical_education_number,type,PROG_YEAR,PROG_GRAD_YR,SCH_PARTY_ID,GRAD_DT,STS_TYPE_ID,DEGREE_CD,...,TRAIN_TYPE,CONF_STS_ID,GME_STS_TYPE_CD,STATE_ID_GME,COUNTRY_ID_GME,ORG_NM_GME,THRU_DT_GME,SPEC_CD,SPEC_ID,PRIM_SPEC_CD
3037,6156511,00518150413,00518150413,Physician,0.0,0.0,2762583,2016-06-05,9,MD,...,R,52.0,INCOMPLETE,256.0,6705.0,Rutgers New Jersey Medical School,NaT,GS,1703,US
4719,1798827,00102760786,00102760786,Physician,0.0,0.0,2701280,1976-01-01,9,MD,...,R,52.0,NO-REPORT,196.0,6705.0,Baptist Health System Inc,NaT,GS,1703,US
4934,1833432,01102811948,01102811948,Physician,0.0,0.0,2778203,1981-05-01,9,MD,...,I,52.0,NO-REPORT,232.0,6705.0,Los Angeles County-Harbor-UCLA Medical Center,NaT,GS,1703,US
8489,1844042,00106830525,00106830525,Physician,0.0,0.0,2855125,1983-06-01,9,MD,...,R,52.0,NO-REPORT,196.0,6705.0,Baptist Health System Inc,NaT,GS,1703,US
12027,2128515,02301900832,02301900832,Physician,0.0,0.0,2762593,1990-05-01,9,MD,...,I,52.0,NO-REPORT,252.0,6705.0,Case Western Reserve University/University Hos...,NaT,GS,1703,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227907,10833405,57503150095,57503150095,Physician,0.0,0.0,5824088,2015-01-01,9,MD,...,R,52.0,COMPLETED,257.0,6705.0,Lehigh Valley Health Network,NaT,TRN,1873,US
1227908,6638187,30813180126,30813180126,Physician,0.0,0.0,2848837,2018-01-01,9,MD,...,R,52.0,COMPLETED,258.0,6705.0,Damas Hospital,NaT,TRN,1873,US
1228351,9252935,49641150061,49641150061,Physician,0.0,0.0,2770568,2015-01-01,9,MD,...,F,52.0,COMPLETED,248.0,6705.0,St Agnes HealthCare,NaT,IAN,1946,US
1228632,8572013,05176180040,05176180040,Physician,0.0,0.0,6654093,2018-05-19,9,DO,...,R,52.0,CURRENT,226.0,6705.0,Mountain State Osteopathic Postdoctoral Traini...,NaT,ONM,1952,US


In [102]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [76]:
ppd = use.get_newest_ppd()

INFO:useful_functions:PPD file has not been updated since Aug 13 2022
INFO:useful_functions:Reading local ppd from Aug 13 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (13,14,20,21,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [95]:
ppd[ppd.FIRST_NAME=="DE'ANDRE"].PRIM_SPEC_CD

237316    OTO
Name: PRIM_SPEC_CD, dtype: object

In [58]:
school_list = ['University','School','Institute','College','Univ']
for thing in ALL.ORG_NM_SCH.unique():
    for word in school_list:
        school_word = False
        if word in thing:
            school_word = True
            break
    if school_word == False:
        print(thing)

Coll Osteo Phys & Surgs Of Los Angeles, Los Angeles (Now Allopathic)
Faculdade de Ciencias Medicas de Santos (UNILUS)
Inst Superior de Ciencias Med Villa Clara 'Dr Serafin Ruiz de Zarate'
Omsk State Medical Academy
Escuela Autonoma de Ciencias Medicas de Centro America
Ruth and Bruce Rappaport Fac of Med, Technion Israel Inst of Technology
Ma Coll Of Osteo,Boston
Tashkent Medical Academy
Mid West Med Coll, Kansas City (See 028-43)
Toronto Sch Of Med, Toronto, Ont, Canada (Inactive)
Jawaharlal Inst of Post-Graduate Medical Education and Research (JIPMER)
Escola Superior de Cien da Santa Casa de Misericordia de Vitoria Fac de Med
Faculdade de Medicina de Jundiai (FMJ)
UNIVAS Faculdade de Ciencias da Saude Dr Joao Antonio Garcia Coutinho
Faculdade de Medicina de Sao Jose do Rio Preto (FAMERP)
Faculdade de Medicina de Valenca (FMV)
Faculdade Evangelica do Parana (FEPAR) Faculdade de Medicina
Escola de Med Souza Marques (EMSM), Fundacao Tecnico Educacao Souza Marques
Fac de Med de Catanduva

In [24]:
bads = pd.read_csv('../../Data/Measurement/bad_countries_new.csv')

In [25]:
bads = bads[bads.DESC!='Unknown']

In [32]:
ALL.columns

Index(['PARTY_ID', 'ME', 'medical_education_number', 'type', 'PROG_YEAR',
       'PROG_GRAD_YR', 'SCH_PARTY_ID', 'GRAD_DT', 'STS_TYPE_ID', 'DEGREE_CD',
       'GRAD_CONF_IND', 'COUNTRY_ID_SCH', 'PARTY_ID_SCHOOL', 'SCHOOL_ID',
       'STATE_ID_SCH', 'PARTY_HOSPITAL_ID_SCH', 'ORG_NM_SCH', 'THRU_DT_SCH',
       'GRAD_DATE', 'COUNTRY', 'STATUS', 'PARTY_HOSPITAL_ID_GME', 'BEGIN_DT',
       'END_DT', 'PRIM_SPEC_ID', 'SEC_SPEC_ID', 'TRAIN_TYPE', 'CONF_STS_ID',
       'GME_STS_TYPE_CD', 'STATE_ID_GME', 'COUNTRY_ID_GME', 'ORG_NM_GME',
       'THRU_DT_GME'],
      dtype='object')

In [48]:
ALL.groupby(['GME_STS_TYPE_CD','type']).count()

PARTY_ID      ME  medical_education_number  \
GME_STS_TYPE_CD type                                                    
COMPLETED       Physician    631463  631463                    631463   
                Resident      21098   21098                     21098   
                Student        2144    2144                      2144   
CURRENT         Physician     14756   14756                     14756   
                Resident      82878   82878                     82878   
                Student        3228    3228                      3228   
INCOMPLETE      Physician     33127   33127                     33127   
                Resident       2157    2157                      2157   
                Student         121     121                       121   
NO-REPORT       Physician    447608  447608                    447608   
                Resident        276     276                       276   
                Student           5       5                         5   

                           PROG_YEAR  PROG_GRAD_YR  SCH_PARTY_ID  GRAD_DT  \
GME_STS_TYPE_CD type                                                        
COMPLETED       Physician     631463        631463        631463   631463   
                Resident       21097         21097         21098    21098   
                Student            0             0          2144     2144   
CURRENT         Physician      14756         14756         14756    14756   
                Resident       82878         82878         82878    82878   
                Student            0             0          3228     3228   
INCOMPLETE      Physician      33127         33127         33127    33127   
                Resident        2157          2157          2157     2157   
                Student            0             0           121      121   
NO-REPORT       Physician     447608        447608        447608   447608   
                Resident         276           276           276      276   
                Student            0             0             5        5   

                           STS_TYPE_ID  DEGREE_CD  GRAD_CONF_IND  ...  \
GME_STS_TYPE_CD type                                              ...   
COMPLETED       Physician       631463     631463         512646  ...   
                Resident         21098      21098          16672  ...   
                Student           2144       2144             11  ...   
CURRENT         Physician        14756      14756           9769  ...   
                Resident         82878      82878          71794  ...   
                Student           3228       3228            189  ...   
INCOMPLETE      Physician        33127      33127          25651  ...   
                Resident          2157       2157           1777  ...   
                Student            121        121              1  ...   
NO-REPORT       Physician       447608     447608         445662  ...   
                Resident           276        276            114  ...   
                Student              5          5              0  ...   

                           BEGIN_DT  END_DT  PRIM_SPEC_ID  SEC_SPEC_ID  \
GME_STS_TYPE_CD type                                                     
COMPLETED       Physician    631463  631463        631463       631463   
                Resident      21098   21098         21098        21098   
                Student        2144    2144          2144         2144   
CURRENT         Physician     14756   14756         14756        14756   
                Resident      82878   82878         82878        82878   
                Student        3228    3228          3228         3228   
INCOMPLETE      Physician     33127   33127         33127        33127   
                Resident       2157    2157          2157         2157   
                Student         121     121           121          121   
NO-REPORT       Physician    447608  447608        447608       447608   
              

PHYS - 6
RES - 5
33127+14756
2190